# Assignation taxonomique utilisant Bash

Ce notebook comprend les étapes d'assignation taxonomique utilisant du code *bash*. Il intervient :
- à la partie 4)B), avant la visualisation des résultats VSEARCH
- à la partie 4)C)2), après préparation de l'arbre de référence

## 1 - Assignation taxonomique VSEARCH

Script et fichiers inputs issus de [metabarcoding_utils](https://gitlab.com/metabarcoding_utils) \
Avant de lancer le script, les fichiers inputs doivent être chargés sur l'espace de travail (`Input/nextflow`):

### A) Pour importation base de données de référence (PR2)
- [lien vers abims.config](https://gitlab.com/metabarcoding_utils/metab-refdb-importer/-/blob/main/conf_templates/abims.config?ref_type=heads)
- [lien vers parameters.yaml](https://gitlab.com/metabarcoding_utils/metab-refdb-importer/-/blob/main/param_templates/pr2_params.yaml?ref_type=heads) \
**/!\ Ouvrir le fichier `parameters.yaml` et préciser où l'on souhaite stocker les données de la base de référence via le paramètre "outdir: 'Output/nextflow'**

Parmis les différents fichiers importés, deux vont être utilisés par la suite :
- séquences de référence : `pr2_version_5.0.0_SSU_TAReuk454FWD1_TAReukREV3.fasta.gz`
- taxonomie de référence : `pr2_version_5.0.0_SSU_taxo.tsv`

### B) Pour assignation taxonomique
- [lien vers abims.config_assign](https://gitlab.com/metabarcoding_utils/taxonomic-assignment/-/blob/main/conf_templates/abims.config?ref_type=heads)
- parameters_assign.yaml : exemple dans [README.md](https://gitlab.com/metabarcoding_utils/taxonomic-assignment/-/blob/main/README.md?ref_type=heads), paragraphe "Generic usage" \
**/!\ Ouvrir `parameters_assign.yaml` et préciser les cheminements vers les fichiers input (séquences query, séquences de référence, taxonomie de référence) \
Séquence query : fichier `ASV_crasp.fasta` (dossier **`Output`**)**

Paramètres supplémentaires (à la suite, toujours dans `parameters_assign.yaml`) :
- best_hit_only: true \
_ne garde que la séquence de référence présentant le plus fort pourcentage de similarité à la séquence query_
- outdir: output directory \
_préciser le dossier où l'on souhaite stocker les résultats de l'assignation taxonomique_
- vsearch_query_cov: 0.97 \
_remplace le default (=1), avec lequel l'une des séquence de Craspedida n'apparaît pas dans les résultats d'assignation (car "query_cov" de la séquence de référence associée légèrement inférieur à 100%)_

### C) Script

In [4]:
module load nextflow/23.04.1

nextflow \
  run https://gitlab.com/metabarcoding_utils/metab-refdb-importer \
  -r main \
  -c Input/nextflow/abims.config \
  -params-file Input/nextflow/parameters.yaml
  
nextflow \
  run https://gitlab.com/metabarcoding_utils/taxonomic-assignment \
  -r main \
  -c Input/nextflow/abims.config_assign \
  -params-file Input/nextflow/parameters_assign.yaml

ERROR ~ .nextflow/history.lock (Permission denied)

 -- Check '.nextflow.log' file for details
ERROR ~ .nextflow/history.lock (Permission denied)

 -- Check '.nextflow.log' file for details


: 1

## 2 - Placement phylogénétique EPA-ng

### A) Préparation de l'environnement de travail

Avant de lancer les analyses, il faut installer les modules suivant :
- [gotree](https://anaconda.org/bioconda/gotree),
- [mafft](https://anaconda.org/bioconda/mafft),
- [epa-ng](https://anaconda.org/bioconda/epa-ng),
- [raxml-ng](https://anaconda.org/bioconda/raxml-ng)

Une fois installés, il faut les importer dans l'environnement de travail :

In [5]:
module load epa-ng
module load mafft
module load raxml-ng

### B) Alignement des séquences

Nous utilisons **mafft** et le commande **-add** pour aligner nos séquences (Craspedida ASVs : `Output/ASV_crasp.fasta`) avec celles utilisées pour créer l'arbre phylogénétique de référence (`Input/18S_EFL_HSP90.concatenated.fasta`).

In [6]:
 mafft \
  --add Output/ASV_crasp.fasta \
  --keeplength Input/18S_EFL_HSP90.concatenated.fasta \
  > Output/all_aligned.fasta

nadd = 10
nthread = 0
nthreadpair = 0
nthreadtb = 0
ppenalty_ex = 0
stacksize: 8192 kb
generating a scoring matrix for nucleotide (dist=200) ... done
Gap Penalty = -1.53, +0.00, +0.00



Making a distance matrix ..

There are 15 ambiguous characters.
    1 / 45
done.

Constructing a UPGMA tree (efffree=0) ... 
   40 / 45
done.

Progressive alignment 1/2... 
STEP    14 / 44 
done.

Making a distance matrix from msa.. 
    0 / 45
done.

Constructing a UPGMA tree (efffree=1) ... 
   40 / 45
done.

Progressive alignment 2/2... 
STEP    17 / 44 
done.

disttbfast (nuc) Version 7.515
alg=A, model=DNA200 (2), 1.53 (4.59), -0.00 (-0.00), noshift, amax=0.0
0 thread(s)


To keep the alignment length, 9 letters were DELETED.
To know the positions of deleted letters, rerun the same command with the --mapout option.

Strategy:
 FFT-NS-2 (Fast but rough)
 Progressive method (guide trees were built 2 times.)

If unsure which option to use, try 'mafft --auto input > output'.
For more information, see 

Suite à la sortie de la fonction précédente, nous avons créé un ficher `all_alligned.fasta` qui contient les alignements de nos séquences requêtes et des séquences de référence. Nous voulons les séparer en 2 fichiers différents en utilisant **eap-ng** et la commande **--split**. Nous aurons en sortie 2 fichiers `query.fasta` (alignement de no séquences requête) et `reference.fasta` (alignement des séquences de référence).

In [7]:
epa-ng \
  --split Input/18S_EFL_HSP90.concatenated.fasta \
  Output/all_aligned.fasta \
  --outdir Output/ \
  --redo

INFO Splitting files based on reference: Input/18S_EFL_HSP90.concatenated.fasta


Nous cherchons d'abors à choisir le modèle d'évolution que nous allons utiliser pour le placement. Pour cela, nous évaluons le meillleur modèle à partir de l'arbre et des alignements de référence en utilisant **raxml-ng**.

In [8]:
raxml-ng \
  --evaluate \
  --msa Input/18S_EFL_HSP90.concatenated.fasta \
  --tree Output/Ref_tree.nwk \
  --prefix Output/info \
  --model GTR+G+F \
  --redo


RAxML-NG v. 1.1 released on 29.11.2021 by The Exelixis Lab.
Developed by: Alexey M. Kozlov and Alexandros Stamatakis.
Contributors: Diego Darriba, Tomas Flouri, Benoit Morel, Sarah Lutteropp, Ben Bettisworth.
Latest version: https://github.com/amkozlov/raxml-ng
Questions/problems/suggestions? Please visit: https://groups.google.com/forum/#!forum/raxml

System: Intel(R) Xeon(R) Gold 5320 CPU @ 2.20GHz, 52 cores, 1007 GB RAM

RAxML-NG was called at 09-Dec-2024 22:44:59 as follows:

raxml-ng --evaluate --msa Input/18S_EFL_HSP90.concatenated.fasta --tree Output/Ref_tree.nwk --prefix Output/info --model GTR+G+F --redo

Analysis options:
  run mode: Evaluate tree likelihood
  start tree(s): user
  random seed: 1733780699
  tip-inner: OFF
  pattern compression: ON
  per-rate scalers: OFF
  site repeats: ON
  branch lengths: proportional (ML estimate, algorithm: NR-FAST)
  SIMD kernels: AVX2
  parallelization: coarse-grained (auto), PTHREADS (auto)


[00:00:00] Reading alignment from file: In

### C) Placement 

Nous avons désormais tous les éléments pour faire le placement avec **epa-ng** :

In [9]:
epa-ng \
  --ref-msa Input/18S_EFL_HSP90.concatenated.fasta \
  --tree Output/Ref_tree.nwk \
  --query Output/query.fasta \
  --model Output/info.raxml.bestModel \
  --outdir Output \
  --redo

INFO Selected: Output dir: Output/
INFO Selected: Query file: Output/query.fasta
INFO Selected: Tree file: Output/Ref_tree.nwk
INFO Selected: Reference MSA: Input/18S_EFL_HSP90.concatenated.fasta
INFO Selected: Automatic switching of use of per rate scalers
INFO Selected: Preserving the root of the input tree
INFO Selected: Specified model file: Output/info.raxml.bestModel
INFO     ______ ____   ___           _   __ ______
        / ____// __ \ /   |         / | / // ____/
       / __/  / /_/ // /| | ______ /  |/ // / __  
      / /___ / ____// ___ |/_____// /|  // /_/ /  
     /_____//_/    /_/  |_|      /_/ |_/ \____/ (v0.3.8)
INFO Using model parameters:
INFO    Rate heterogeneity: GAMMA (4 cats, mean),  alpha: 0.390902 (user),  weights&rates: (0.25,0.015424) (0.25,0.174928) (0.25,0.721245) (0.25,3.0884) 
        Base frequencies (empirical): 0.257649 0.169648 0.27156 0.301144 
        Substitution rates (user): 1.51618 2.56344 1.21118 2.02544 4.72887 1
INFO Output file: Output/epa_

Le placement phylogénétique a été effectué. Pour la visualisation, il faut maintenant continuer dans le notebook `main_notebook-R.ipynb` partie ***4)C)3) Visualisation des placements phylogénétiques***